# AgentForce SDK - Create Agent Examples

This notebook demonstrates different ways to create agents using the AgentForce SDK.

## Prerequisites

First, let's install the AgentForce SDK:

In [ ]:
# Install the latest version of AgentForce SDK
!pip install agentforce-sdk

## Setup

Let's set up our credentials for AgentForce:

In [ ]:
import os
from agent_sdk import Agentforce, AgentUtils
from agent_sdk.models.agent import Agent
from agent_sdk.models.topic import Topic
from agent_sdk.models.action import Action
from agent_sdk.models.input import Input
from agent_sdk.models.output import Output
from agent_sdk.models.system_message import SystemMessage
from agent_sdk.models.variable import Variable
from agent_sdk.core.auth import BasicAuth
from google.colab import userdata

# Replace with your Salesforce credentials
username = userdata.get("SF_USERNAME")
password = userdata.get("SF_PASSWORD")

auth = BasicAuth(username=username, password=password)

# Initialize the AgentForce client
agentforce = Agentforce(auth=auth)

/tmp/ipython-input-2-478862090.py:6: DeprecationWarning: Importing Input from agent_sdk.models.input is deprecated. Please import from agent_sdk.models.action or agent_sdk.models instead.
  from agent_sdk.models.input import Input
/tmp/ipython-input-2-478862090.py:7: DeprecationWarning: Importing Output from agent_sdk.models.output is deprecated. Please import from agent_sdk.models.action or agent_sdk.models instead.
  from agent_sdk.models.output import Output
INFO:agent_sdk.core.agentforce:Initializing Agentforce SDK
INFO:agent_sdk.core.agentforce:Running in development environment


## Method 1: Create Agent from JSON File

This is the simplest way to create an agent, by using a pre-defined JSON file.

In [ ]:
# Path to JSON file
json_file_path = "/content/input.json"

# Create the agent from JSON file
agent = AgentUtils.create_agent_from_file(json_file_path)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Company: {agent.company_name}")
print(f"Topics: {len(agent.topics)}")

# To deploy the agent:
result = agentforce.create(agent)
print(f"Agent created successfully")

# To deploy the agent:
result = agentforce.create(agent)
print(f"Agent created successfully")

## Method 2: Create Agent from Directory Structure

This method uses a modular directory structure where each component is defined in a separate file.

In [ ]:
# Path to directory structure
directory_path = "../assets/chained_agent_dir"
agent_name = "order_management_agent"

# Create the agent from directory structure
agent = AgentUtils.create_agent_from_directory_structure(directory_path, agent_name)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Topics: {len(agent.topics)}")

# Display topic information
for i, topic in enumerate(agent.topics):
    print(f"\nTopic {i+1}: {topic.name}")
    print(f"Description: {topic.description}")
    print(f"Actions: {len(topic.actions) if topic.actions else 0}")

    if topic.actions:
        for j, action in enumerate(topic.actions):
            print(f"  Action {j+1}: {action.name}")

# To deploy the agent:
# result = agentforce.create(agent)
# print(f"Agent created successfully")

## Method 3: Create Agent Programmatically

This method demonstrates how to create an agent by constructing it programmatically with objects.

In [ ]:
# Create an action with inputs and outputs
action = Action(
    name="findOrder",
    description="Find order details using an order ID",
    inputs=[
        Input(
            name="orderID",
            description="Order identification number",
            data_type="string",
        )
    ],
    outputs=[
        Output(
            name="orderDetails", description="Details of the order", data_type="object"
        )
    ],
)

# Create a topic with the action
topic = Topic(
    name="Order Management",
    description="Handles all user requests related to finding and managing orders",
    scope="public",
    instructions=[
        "If a user cannot find their order, attempt to locate it using the order ID",
        "If a user wants to check the status of their order, retrieve the order details",
    ],
    actions=[action],
)

# Create the agent with the topic
agent = Agent(
    name="ProgrammaticAgent",
    description="An agent created programmatically for order management",
    agent_type="External",
    agent_template_type="EinsteinServiceAgent",
    company_name="Example Corp",
    sample_utterances=["What's the status of my order?", "I need to find my order"],
    system_messages=[
        SystemMessage(message="Welcome to Order Management!", msg_type="welcome"),
        SystemMessage(message="I'm sorry, I encountered an error.", msg_type="error"),
    ],
    variables=[Variable(name="apiKey", data_type="Text", var_type="conversation", visibility="Internal", developer_name="apiKey", label="API Key")],
    topics=[topic],
)

# View the agent configuration
print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Topics: {len(agent.topics)}")

# To deploy the agent:
result = agentforce.create(agent)
print(f"Agent created successfully")

INFO:agent_sdk.core.agentforce:Deploying agent: ProgrammaticAgent
INFO:agent_sdk.core.agentforce:Generating metadata files...


Agent Name: ProgrammaticAgent
Description: An agent created programmatically for order management
Topics: 1


INFO:agent_sdk.core.agentforce:Metadata generation completed successfully
INFO:agent_sdk.core.agentforce:Creating deployment zip file...
INFO:agent_sdk.core.agentforce:Deployment zip file created successfully
INFO:agent_sdk.core.agentforce:Initiating deployment to Salesforce...
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:No BotVersion found for Bot with DeveloperName 'ProgrammaticAgent'
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:No Planner Definition Found with 'ProgrammaticAgent_v1'
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:No in-progress deployments found.
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:Deploying agent ProgrammaticAgent to Salesforce
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:Deployment started with ID: 0AfHs00002xaoBeKAI
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:Checking deployment status for ID: 0AfHs00002xaoBeKAI?includeDetails=true
INFO:agent_sdk.core.deploy_tools.deploy_zipfile:Status check attempt 1 of 30
INFO:agent_sdk.core.deploy_too

Agent created successfully


## Method 4: Create & Export Agent Directory Structure

This method demonstrates how to create an agent programmatically and then export it to a directory structure.

In [ ]:
# We'll use the same agent we created programmatically above
export_dir = "/content/exported_agent"

# Create the directory structure
AgentUtils.create_agent_directory_structure(export_dir, agent)

print(f"Agent exported to directory: {export_dir}")

# You can then read it back
# reimported_agent = AgentUtils.create_agent_from_directory_structure(export_dir, "ProgrammaticAgent")
# print(f"Re-imported agent name: {reimported_agent.name}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Method 5: Create Agent with Dependent Metadata

This method demonstrates how to create an agent that uses  additional dependent metadata (like Apex classes, permission sets, and custom objects) that Agent depends on. The example shows how to create an order management agent using a dependent Apex class action and a permission set.

In [ ]:
from agent_sdk.models.action import Action, Input, Output
from agent_sdk.models.topic import Topic
from agent_sdk.models.system_message import SystemMessage

def create_order_management_topic() -> Topic:
    """Create the Order Management topic with its actions."""
    topic = Topic(
        name="Order Management",
        description="Handle order-related queries using custom Apex classes",
        scope="Handle order-related queries and actions",
    )

    topic.instructions = [
        "Always ask for the customer Id and Order Id before answering query about the order status",
        "If a user requests the status of an order, provide a detailed update on the order's current status, including shipping status by using the GetOrderInfo action.",
        "When the user provides the Customer Id and Order Id use and call GetOrderInfo action and display the results from the action in user friendly readable format",
        "If the user does not provide Customer Id and Order Id, ask them to provide one."
    ]

    # Create action that will use custom Apex class
    get_order_info_action = Action(
        name="GetOrderInfo",
        invocation_target="OrderManagementService",
        invocation_target_type="apex",
        description="Get order information using custom Apex class",
        inputs=[
            Input(
                name="orderId",
                description="ID of the order to check",
                data_type="String",
                required=True,
            ),
            Input(
                name="customerId",
                description="ID of the customer",
                data_type="String",
                required=True,
            )
        ],
        outputs=[
            Output(
                name="status",
                description="Status of the order info retrieval",
                data_type="String",
                required=True,
            ),
            Output(
                name="orderDetails",
                description="Details of the order",
                data_type="String",
                required=True,
            )
        ],
        example_output={"status": "success", "order_details": "Order #12345: 2 items, Status: Processing"}
    )

    topic.actions = [get_order_info_action]
    return topic

# Create the agent with dependent metadata
agent = Agent(
    name="Order Management",
    description="An agent that uses custom Apex classes to manage orders",
    agent_type="External",
    company_name="Salesforce",
)

# Set sample utterances
agent.sample_utterances = [
    "What's the status of my order?",
    "Can you check order #12345?",
    "Show me my order details",
    "Get information about my recent order",
]

# Set system messages
agent.system_messages = [
    SystemMessage(
        message="You are an order management assistant.",
        msg_type="system"
    ),
    SystemMessage(
        message="Always verify order IDs before retrieving information.",
        msg_type="system"
    ),
]

# Set topics
agent.topics = [create_order_management_topic()]

# Specify the path to dependent metadata directory
metadata_dir = "https://drive.google.com/drive/folders/1vYi5Xpw8dg5xMseUrFxs6woHAdTXQjWd?usp=sharing"

# Create agent with dependent metadata
result = agentforce.create(agent, dependent_metadata_dir=metadata_dir)

print(f"Agent Name: {agent.name}")
print(f"Description: {agent.description}")
print(f"Topics: {len(agent.topics)}")
print(f"Deployment Status: {result.get('deployResult', {}).get('status')}")
print(f"Deployment ID: {result.get('id', 'N/A')}")

### Understanding Dependent Metadata

The dependent metadata feature allows you to provide your own custom Salesforce metadata (Apex classes, flows, objects, etc.) instead of using the default template classes. In this example:

1. We created an agent that uses a custom Apex class (`OrderManagementService`) for order management
2. The custom metadata is located in the `dependent_metadata_dir` directory
3. The metadata directory must contain:
   - `package.xml` - Required metadata package definition
   - Custom Apex classes with proper annotations
   - Any other required Salesforce metadata

When deploying the agent, the SDK will:
1. Validate the dependent metadata directory
2. Include the custom metadata in the deployment package
3. Deploy both the agent configuration and custom metadata to Salesforce